In [12]:
from pycparser import parse_file, c_ast

global parents, variables
parents={}

In [13]:
class Condition:
    def __init__(self, condition):
        self.condition = condition
        (self.left_operand, self.operator, self.right_operand) = self.handleCondition(condition)
    
    def handleCondition(self, condition):
        left_operand = condition.left
        right_operand = condition.right
        operator = condition.op
        
        if operator == ">":
            return (right_operand, "<", left_operand)
        elif operator == ">=":
            return (right_operand, "<=", left_operand)
        else:
            return (left_operand, operator, right_operand)

class Context:  
    def __init__(self, condition = None, bool_condition = True, next_nodes = [],\
                 previous_context = None):
        if previous_context is not None:
            self.conditions = previous_context.conditions.append((condition, bool_condition))
            self.variables = previous_context.variables
            self.nexts = previous_context.nexts + next_nodes
            self.checkCondition()
        else:
            self.conditions = []
            self.variables = {}
            self.nexts = []
    
    def addVariable(self, node_decl):
        name = node_decl.name
        types = node_decl.type.type.names
        if 'int' not in types:
            raise 'Only int type allowed'
        if 'unsigned' in types:
            return
            
    def pop(self):
        if self.nexts != []:
            return self.nexts.pop()
        return None
        
    def checkCondition(self):
        """
        Si self.state = True condition possible
        Si self.state = False condition impossible donc le branchement est inutile
        """
        
        # simplification des deux listes représentant les variables
        # propagation des singletons grâce à leur valeur trouvée dans la bibliothèque
        # si il y a encore des variables entièrement non définies de chaque côté -> 
        # deux contexte et pas d'update
        # si non définie d'un seul côté -> on a les deux contextes et update de la valeur 
        # de la variable
        # sinon -> un seul contexte et pas d'update
        self.state = True
        return

class Singleton():
    """
        Représente un singleton
        
        Attributs
        ----------
        value: @Variable 
            valeur du singleton
    """
    def __init__(self, value):
        self.value = value

class Interval():
    """
        Représente un interval
        
        Attributs
        ----------
        min: @Value
            Minimum de l'intervalle
        max: @Value 
            Maximum de l'intervalle
    """
    def __init__(self, minimum, maximum):
        self.min = minimum
        self.max = maximum

class Variable():
    """
        Représente une variable
        
        Attributs
        ----------
        name: String 
            Nom de la varialbe
        intervals: list[@Interval] 
            Ensemble d'intervalles pour la variable
    """
    def __init__(self, name, intervals):
        self.name = name
        self.intervals = interval

class Value():
    """
        Représente une valeur de variable qui est sous la forme: @Variable + Constante
        
        Attributs
        ----------
        constante: Int
            Constante
        variable: @Variable
            Variable
    """
    def __init__(self, constante = 0, variable = None):
        self.variable = variable
        self.constante = constante
        
class Operation():
    def __init__(self, left, operator, right):
        self.left = left
        self.operator = operator
        self.right = right
    
    def computeValue(self, left, operator, right):
        value_computed = Value()
        if(operator == "+"):
            value_computed.constante = left.constante + right.constante
            if left.variable is None:
                value_computed.variable = right.variable
                value_computed.operator = right.operator
            else:
                list_variable_left = left.variable.copy()
                value_computed.variable = list_variable_left.extends(\
                                                ['+'] + right.variable)
        elif(operator == "-"):
            value_computed.constante = left.constante - right.constante
            if left.variable is None:
                value_computed.variable = right.variable
                value_computed.operator = right.operator
            else:
                list_variable_left = left.variable.copy()
                value_computed.variable = list_variable_left.extends(\
                                                ['-'] + right.variable)       
        return value_computed
    
    def compute(self, left, operator, right)
        if type(left) is Interval && type(right) is Interval:
            return Interval(self.computeValue(left.min, operator, right.min),\
                            self.computeValue(left.max, operator, right.max))
        elif type(left) is Singleton && type(right) is Interval:
            return Interval(self.computeValue(left, operator, right.min),\
                            self.computeValue(left, operator, right.max))
        elif type(left) is Interval && type(right) is Singleton:
            return Interval(self.computeValue(left.min, operator, right),\
                            self.computeValue(left.max, operator, right)) 
        else:
            return Singleton(self.computeValue(left, operator, right))

In [14]:
def parse_c_file(file):
    ast = parse_file(file, use_cpp=True,
        cpp_path='gcc',
        cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return ast

In [15]:
ast_graph = parse_c_file('./C files/if.c')

In [16]:
def computeParents(node, parent):
    global parents
    parents[node] = parent
    for (child, value) in node.children():
        computeParents(value, node) 

In [17]:
computeParents(ast_graph, None)

In [18]:
def isIf(node):
    return type(node) == c_ast.If

In [19]:
def getCoord(node):
    return node.coord

In [20]:
def isDeclaration(node):
    return type(node) == c_ast.Decl

In [21]:
def isAssignment(node):
    return type(node) == c_ast.Assignment

In [22]:
def visitNode(context):
    node = context.pop()
    if node is not None:
        if isIf(node):
            condition = Condition(node.cond)
            context1 = Context(condition = condition, bool_condition = True,\
                               next_nodes = [node.iftrue], previous_context = context)
            context2 = Context(condition = condition, bool_condition = False,\
                               next_nodes = [node.iffalse], previous_context = context)
            if context1.state and context2.state:
                return visitNode(context1) + visitNode(context2)
            elif context1.state and not context2.state:
                return visitNode(context1) + [context2]
            elif not context1.state and context2.state:
                return visitNode(context2) + [context1]
            else:
                return [context1, context2]
        elif isDeclaration(node):
            context.addVariable(node)
        elif isAssignment(node):
            context.assign(node)
    return []

In [23]:
def visitGraph(graph):
    context = Context(next_nodes = graph.children())
    return visitNode(context)